In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import random
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
random.seed(1)

In [4]:
# set data path
train_dir_path = "./2021-ai-w10-p2/images/images"
test_dir_path = "./2021-ai-w10-p2/test_data/test_data"
submission_csv_path = "./2021-ai-w10-p2/format.csv"

In [5]:
# load train data 
from tqdm import tqdm
import os
from PIL import Image


def load_train_data(dir_path):
    train_y = []
    train_x = []
    for dir_name in tqdm(os.listdir(dir_path)):
        for img_name in os.listdir(os.path.join(dir_path, dir_name)):
            img_path = os.path.join(dir_path, dir_name, img_name)
            img = np.array(Image.open(img_path)).transpose((2, 0, 1))
            # Scaling
            img = img / np.max(img)
            train_x.append(img)
            train_y.append(int(dir_name))
    return np.array(train_x), np.array(train_y)


train_x, train_y = load_train_data(train_dir_path)
print(train_x.shape, train_y.shape)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.18s/it]


(50000, 3, 32, 32) (50000,)


In [6]:
# set data on Tensor and Dataset
train_x = torch.Tensor(train_x).to(device)
train_y = torch.LongTensor(train_y).to(device)
train_dataset = torch.utils.data.TensorDataset(train_x, train_y)

In [7]:
model = torchvision.models.googlenet(pretrained=True)

# freezing
# for param in model.parameters():
#     param.requires_grad = False

model.fc = torch.nn.Linear(in_features=1024, out_features=10, bias=True).to(device)

torch.nn.init.xavier_normal_(model.fc.weight)

model = model.to(device)
# print(model)

In [8]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 50
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
epochs = 10
loss = torch.nn.CrossEntropyLoss()

In [9]:
def train_model(model, data_loader, optim):
    sum_cost = 0
    sum_acc = 0
    for x, y in data_loader:
        model.train()
        output = model(x)
        cost = loss(output, y)
        optim.zero_grad()
        cost.backward()
        optim.step()
        sum_cost += cost.item()
        model.eval()
        predict = model(x)
        predict = torch.argmax(predict,dim=1)
        acc = predict == y
        sum_acc += acc.sum().item()
    return sum_cost,sum_acc

In [10]:
# learning
for epoch in range(epochs + 1):
    cost, acc = train_model(model, train_data_loader, optim)
    cost = cost / len(train_data_loader.dataset)
    acc = acc/ len(train_data_loader.dataset)  
    print(epoch, cost,acc)

c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 0.020579560912251472 0.7145
1 0.013924890954494477 0.81046
2 0.012882868792712688 0.82186
3 0.01015676796913147 0.8724
4 0.008213066135197879 0.90086
5 0.007196333781629801 0.91646
6 0.0059139532723277806 0.93612
7 0.005037943969741464 0.95214
8 0.004373461634740234 0.95514
9 0.003924567566923797 0.9641
10 0.003529870397038758 0.96576


In [11]:
with torch.no_grad():
    model.eval()
    predict = model(train_x)
    predict = torch.argmax(predict, dim=1)
    acc = train_y == predict
    acc = acc.float().mean().cpu().detach().item()
print(acc)

0.9537599682807922


In [12]:
# load test data 
def load_test_data(dir_path):
    test_x = []
    img_names = os.listdir(dir_path)
    img_names = [int(img_name.split(".")[0]) for img_name in img_names]
    sorted_img_names = sorted(img_names)
    for img_name in tqdm(sorted_img_names):
        img_path = os.path.join(dir_path, str(img_name)) + ".png"
        img = np.array(Image.open(img_path)).transpose((2, 0, 1))
        # Scaling
        img = img / np.max(img)
        test_x.append(img)
    return np.array(test_x)


test_x = load_test_data(test_dir_path)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4177.11it/s]


(10000, 3, 32, 32)


In [13]:
# setd test data on Tensor and Dataset
test_x = torch.Tensor(test_x).to(device)
test_dataset = torch.utils.data.TensorDataset(test_x)
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
def test_model(model, data_loader):
    predicts = []
    for x in tqdm(data_loader):
        predict = model(x[0])
        predict = torch.argmax(predict, dim=1)
        predicts += np.array(predict.cpu().detach()).tolist()
    return np.array(predicts)

In [15]:
# submit
submission = pd.read_csv(submission_csv_path)
with torch.no_grad():
    model.eval()
    predict = test_model(model, test_data_loader)
    submission['label'] = predict
submission.to_csv("submission.csv", index=False)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 131.67it/s]
